# Navigational Model Using Single Batch 1 + Batch 2

In [1]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from ktrain import text
import random
import warnings
from sklearn.utils import shuffle

# Set random seed
random.seed(18)
seed = 18

# Ignore warnings
warnings.filterwarnings('ignore')

# Display options
pd.set_option('display.max_colwidth', None)

## 1. Loading the data and quick exploratory data analysis

In [2]:
import os

# Define the folder path and themes
folder_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/single_theme_using_jaccard_method'
themes = [
    'Aspirational', 'Attainment', 'Community Consciousness', 'Familial', 'Filial Piety', 
    'First Gen', 'Navigational', 'Perseverance', 'Resistance', 'Social', 'Spiritual'
]

# Initialize an empty dictionary to store DataFrames
theme_dataframes = {}
# Loop through each theme and load its corresponding file
for theme in themes:
    # Construct the filename without modifying the theme name
    file_name = f"{theme}_sentence_level_batch_1_jaccard.csv"
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the file exists before attempting to load
    if os.path.exists(file_path):
        theme_dataframes[theme] = pd.read_csv(file_path)
        print(f"Loaded {file_name}")
    else:
        print(f"File not found for theme: {theme}")

Loaded Aspirational_sentence_level_batch_1_jaccard.csv
Loaded Attainment_sentence_level_batch_1_jaccard.csv
Loaded Community Consciousness_sentence_level_batch_1_jaccard.csv
Loaded Familial_sentence_level_batch_1_jaccard.csv
Loaded Filial Piety_sentence_level_batch_1_jaccard.csv
Loaded First Gen_sentence_level_batch_1_jaccard.csv
Loaded Navigational_sentence_level_batch_1_jaccard.csv
Loaded Perseverance_sentence_level_batch_1_jaccard.csv
Loaded Resistance_sentence_level_batch_1_jaccard.csv
Loaded Social_sentence_level_batch_1_jaccard.csv
Loaded Spiritual_sentence_level_batch_1_jaccard.csv


In [2]:
merged_navigational_df_batch_1 = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/merged_Navigational_sentence_level_batch_1_jaccard.csv", encoding='utf-8')
merged_navigational_df_batch_2 = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/Navigational Plus_sentence_level_batch_2_jaccard.csv", encoding='utf-8')

merged_navigational_df = pd.concat([merged_navigational_df_batch_1, merged_navigational_df_batch_2])


seed = 18
# Shuffle the merged dataset
merged_navigational_df = shuffle(merged_navigational_df, random_state=seed)

# # Train-test split
training_df, test_df = train_test_split(merged_navigational_df, test_size=0.1, random_state=18, stratify=merged_navigational_df['label'])

training_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

print("Training and test sets loaded.")

Training and test sets loaded.


In [5]:
print(f"Training dataset shape: {training_df.shape} \nTest dataset shape: {test_df.shape}")
pos_labels = len([n for n in training_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(training_df['label']), (pos_labels/len(training_df['label']))*100))
pos_labels = len([n for n in test_df['label'] if n==1])
print("Positive labels present in the test dataset : {}  out of {} or {}%".format(pos_labels, len(test_df['label']), (pos_labels/len(test_df['label']))*100))

Training dataset shape: (7381, 3) 
Test dataset shape: (821, 3)
Positive labels present in the dataset : 430  out of 7381 or 5.825768866007316%
Positive labels present in the test dataset : 48  out of 821 or 5.84652862362972%


In [6]:
print(training_df.shape)
print(test_df.shape)

(7381, 3)
(821, 3)


## 2. Experimental Design

In [7]:
MAXLEN = 150

In [8]:
X = training_df['sentence']
y = training_df['label']

In [9]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18, stratify=y)
X_test.shape

(1477,)

In [10]:
model_name = 'bert-base-uncased'

In [11]:
distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 40
	99percentile : 54


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 39
	99percentile : 55


In [12]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define classes and class labels
classes = np.array([0, 1])
class_labels = list(training_df.label)

# Compute class weights

class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=class_labels)

# Print class weights
print(class_weights)

class_weights = dict(zip(classes, class_weights))

[0.5309308  8.58255814]


In [13]:
# Build BERT model
# model = text.text_classifier('distilbert', train_data=(X_train, y_train), preproc=distillbert_transformer)
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=16)
# learner.fit_onecycle(2e-5, 4, class_weight=class_weights)
# learner.autofit(2.27E-06, early_stopping=4)
# distillbert_learner.set_weight_decay(0.001)
distillbert_learner.autofit(3.06E-05, early_stopping=4, class_weight=class_weights)
# distillbert_learner.set_weight_decay(0.001)
# distillbert_learner.autofit(2.27E-06, early_stopping=4, class_weight=class_weights)

reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 3.06e-05...
Epoch 1/1024
369/369 [==============================] - 245s 628ms/step - loss: 0.5749 - accuracy: 0.6997 - val_loss: 0.4837 - val_accuracy: 0.7921
Epoch 2/1024
369/369 [==============================] - 231s 625ms/step - loss: 0.4291 - accuracy: 0.8298 - val_loss: 0.4361 - val_accuracy: 0.8497
Epoch 3/1024
369/369 [==============================] - 225s 607ms/step - loss: 0.3661 - accuracy: 0.8340 - val_loss: 0.2995 - val_accuracy: 0.8747
Epoch 4/1024
369/369 [==============================] - 226s 612ms/step - loss: 0.2957 - accuracy: 0.8731 - val_loss: 0.2876 - val_accuracy: 0.8903
Epoch 5/1024
369/369 [==============================] - 227s 614ms/step - loss: 0.2180 - accuracy: 0.8948 - val_loss: 0.2511 - val_accuracy: 0.9086
Epoch 6/1024
369/369 [==============================] - 229s 618ms/step - loss: 0.2173 - accuracy: 0.8870 - val_loss: 0.2932

In [14]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

47/47 [==============================] - 23s 393ms/step
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      1391
           1       0.43      0.56      0.49        86

    accuracy                           0.93      1477
   macro avg       0.70      0.76      0.73      1477
weighted avg       0.94      0.93      0.94      1477



array([[1328,   63],
       [  38,   48]])

In [15]:
distillbert_learner.model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [17]:
distillbert_test_data = test_df['sentence'].tolist()
distillbert_test_label = test_df['label'].tolist()

In [18]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [19]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [20]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 731, False Positive: 42, False Negative: 23, True Positive: 25


In [21]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.95      0.96       773
           1       0.37      0.52      0.43        48

    accuracy                           0.92       821
   macro avg       0.67      0.73      0.70       821
weighted avg       0.93      0.92      0.93       821
 



In [24]:
# distillbert_predictor.save('../../model/first_generation_distilbert_base_uncased_model_10102020') # 256 MB

In [33]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.7419126719027413
